# Análise de Atendimentos nas Unidades Municipais de Saúde de Curitiba

Utilizando o conjunto de dados "Sistema E-Saude - Perfil de atendimento nas Unidades Municipais de Saúde de Curitiba", disponível em http://www.curitiba.pr.gov.br/dadosabertos/consulta/?grupo=1

O dicionário de dados também encontra-se disponível no link acima

## ChangeLog

10/11/2016 - Versão inicial

08/09/2018 - Atualização do dataset e atualizações menores

## Importação, Limpeza dos Dados e Feature Engineering

In [ ]:
import datetime as dt
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline

# Formatação de 2 casas decimais
pd.options.display.float_format = '{:.2f}'.format

# Para mostrar todas as colunas ao dar um 'head'
pd.set_option('display.max_columns', 100)

sns.set()

In [ ]:
# Importando arquivo

df = pd.read_csv(
    filepath_or_buffer = '../input/2018-08-13_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv',
    sep = ';', 
    parse_dates = True,
    encoding = 'ISO-8859-1', 
    low_memory = False
) 

Visualizando alguns registros importados...

In [ ]:
df.head(5)

In [ ]:
print(df.shape[0],'linhas e',df.shape[1],'colunas')

In [ ]:
df.info()

Com base nas informações acima, vamos converter os campos datetime para o formato apropriado.

In [ ]:
df['Data do Atendimento']  = pd.to_datetime(df['Data do Atendimento'], format='%d/%m/%Y %H:%M:%S')
df['Data de Nascimento']   = pd.to_datetime(df['Data de Nascimento'], format='%d/%m/%Y %H:%M:%S')
df['Data do Internamento'] = pd.to_datetime(df['Data do Internamento'], format='%d/%m/%Y %H:%M:%S')

In [ ]:
dataInicial = df['Data do Atendimento'].min().date()
dataFinal = df['Data do Atendimento'].max().date()
print('Atendimentos entre',dataInicial.strftime('%d/%m/%Y'), 'e', dataFinal.strftime('%d/%m/%Y'))
print('Tamanho do intervalo:', (dataFinal-dataInicial).days,'dias')

## Limpeza dos dados

Vamos remover algumas colunas que contém 'Código' e 'Descrição', neste caso deixando apenas as descrições. Vamos deixar o 'Código do CID' para análise futura.

Em relação a quantidades de medicamentos, como não há o nome do medicamento nem a dosagem do mesmo, fica difícil realizar qualquer análise mais aprofundada, desta forma também iremos excluir estas colunas.

A coluna 'Área de Atuação' também possui muitos valores missing e por hora também será excluída.

In [ ]:
# Removendo colunas que não iremos utilizar nesta análise
df.drop(
    [
        'Código do Tipo de Unidade',
        'Código da Unidade',
        'Código do Procedimento',
        'Código do CBO',
        'Qtde Prescrita Farmácia Curitibana',
        'Qtde Dispensada Farmácia Curitibana',
        'Qtde de Medicamento Não Padronizado',
        'Área de Atuação'
    ], axis=1, inplace=True)

Neste dataset, há cerca de 280 linhas sem constar o Código do CID. Vamos remover estas linhas.

In [ ]:
df = df[df['Código do CID'].notnull()]

A coluna 'Município' consta no dataset como 'Municício'. Vamos corrigir.

In [ ]:
df = df.rename(columns={'Municício': 'Município'})

## Criação de novas colunas

Vamos criar as colunas 'Idade' que será baseada 'Data de Atendimento' e 'Data de Nascimento' do paciente. Também vamos criar a coluna 'Classificação Etária' que possuirá os valores 'Criança', 'Adolescente', 'Adulto' e 'Idoso'.

In [ ]:
df['Idade'] = (df['Data do Atendimento'] - df['Data de Nascimento']).dt.days / 365
df['Idade'] = df['Idade'].astype(int)

In [ ]:
def classificacao_etaria(idade):
    if idade < 12:
        return 'Criança'
    elif idade < 18:
        return 'Adolescente'
    elif idade < 65:
        return 'Adulto'
    else:
        return 'Idoso'
    
df['Classificação Etária'] = df['Idade'].apply(classificacao_etaria)

Da mesma forma como fizemos com a 'Idade', vamos criar colunas para saber a data da semana em que o atendimento foi realizado assim como o 'Turno do Atendimento' e se era ou não fim de semana.

In [ ]:
df['Dia da Semana Atendimento'] = df['Data do Atendimento'].dt.dayofweek
df['Dia da Semana Atendimento Descricao'] = df['Data do Atendimento'].dt.weekday_name
df['Hora Atendimento'] = df['Data do Atendimento'].dt.hour
df['Fim de Semana'] = df['Dia da Semana Atendimento Descricao'].isin(["Saturday", "Sunday"])

In [ ]:
def turno_do_atendimento(hora):
    if hora < 6:
        return 'Madrugada'
    elif hora < 19:
        return 'Dia'
    else: 
        return 'Noite'
    
df['Turno do Atendimento'] = df['Hora Atendimento'].apply(turno_do_atendimento)

Vamos verificar se até agora todas as colunas estão com os tipos de dados adequados ou se será necessário definir o tipo explicitamente.

In [ ]:
df.info()
df.head()

# Análise Exploratória

Muitas questões que queremos analisar são respectivos aos atendimentos que geraram internações. Para isto, vamos criar um dataframe separado.

In [ ]:
df_internacao = df[df['Desencadeou Internamento'] == 'Sim']

### Qual a evolução de atendimentos e internações por dia e Tipo de Unidade?

In [ ]:
df_atendimentos_por_dia = df.groupby([df['Data do Atendimento'].dt.date, df['Tipo de Unidade']]).size().reset_index(name="Total Atendimentos")
df_internacoes_por_dia =  df_internacao.groupby([df_internacao['Data do Internamento'].dt.date]).size().reset_index(name="Total Internações")

In [ ]:
df_atendimentos_por_dia['Data do Atendimento']  = pd.to_datetime(df_atendimentos_por_dia['Data do Atendimento'], format= '%Y-%m-%d')
df_internacoes_por_dia['Data do Internamento']  = pd.to_datetime(df_internacoes_por_dia['Data do Internamento'], format= '%Y-%m-%d')

In [ ]:
# Convertendo de long-format para wide-format para plotagem da série temporal
df_atendimentos_por_dia_wide = df_atendimentos_por_dia.pivot(index='Data do Atendimento', columns='Tipo de Unidade', values='Total Atendimentos')

# Substituindo 'nan' por 0
df_atendimentos_por_dia_wide = df_atendimentos_por_dia_wide.fillna(0)

In [ ]:
df_atendimentos_por_dia_wide.head()

In [ ]:
# Definindo índice para plotagem da série temporal
df_internacoes_por_dia.set_index('Data do Internamento', inplace=True)

In [ ]:
df_internacoes_por_dia.head()

In [ ]:
df_atendimentos_por_dia_wide.plot(figsize=(20,10), fontsize=20, linewidth=4)
plt.xlabel('Data do Atendimento', fontsize=20);

In [ ]:
df_internacoes_por_dia.plot(figsize=(20,10), fontsize=20, linewidth=4)
plt.xlabel('Data da Internação', fontsize=20);

### Qual a proporção de atendimentos que desencadeiam internação?

In [ ]:
sns.countplot(df['Desencadeou Internamento'])
print(df['Desencadeou Internamento'].value_counts())

In [ ]:
df['Desencadeou Internamento'].value_counts()[1] / df['Desencadeou Internamento'].value_counts()[0]

** Nem 1% dos atendimentos geram internação. O dataset está desbalanceado e posteriormente deverá ser tratado caso formos utilizar Machine Learning para tentar prever esta variável **

### Qual a proporção de Atendimentos encaminhados para Especialistas e com Solicitação de Exames?

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Encaminhamento para Atendimento Especialista?')
sns.countplot(df['Encaminhamento para Atendimento Especialista'], ax=ax[0])

# Internados
ax[1].set_title('Solicitação de Exames?')
sns.countplot(df['Solicitação de Exames'], ax=ax[1])

### Atendimentos de cidadãos curitibanos X cidadãos não-curitibanos

In [ ]:
proporcao_curitiba = math.trunc(len(df[df['Município'] == 'CURITIBA']) / len(df) * 100)
print(proporcao_curitiba,'% são de atendimentos referentes a cidadãos curitibanos')

Quais municípios mais trazem cidadãos para serem atendidos em Curitiba?

In [ ]:
# Somente os 20 principais municípios
df['Município'].value_counts().reset_index(name='Atendimentos').head(20)

Considerando que 'Colombo' é o município que mais possui cidadãos sendo atendido em Curitiba, em quais lugares estas pessoas passam por atendimento?

In [ ]:
# Somente as 10 principais unidades
df[df['Município'] == 'COLOMBO']['Descrição da Unidade'].value_counts().reset_index(name='Atendimentos').head(10)

'UPA BOA VISTA' que absorve estes atendimentos, o que faz sentido pela sua localização geográfica, ficando cerca de 13 km de distância do local: https://www.google.com/maps/dir/Colombo+-+Boa+Vista,+Colombo+-+PR/UPA+24h+Boa+Vista+-+Avenida+Paran%C3%A1+-+Boa+Vista,+Curitiba+-+PR/@-25.3555923,-49.26806,12.5z/data=!4m13!4m12!1m5!1m1!1s0x94dce81bb32f665f:0x71a27a3f0e78b715!2m2!1d-49.2266344!2d-25.2929784!1m5!1m1!1s0x94dce66f6ff5ece5:0x1b92d4f7158c1e76!2m2!1d-49.2328771!2d-25.3857065

Nesta UPA BOA VISTA, qual o percentual de atendimentos de curitibanos X não-curitibanos?

In [ ]:
df_boa_vista = df[df['Descrição da Unidade'] == 'UPA BOA VISTA']
proporcao_boa_vista_curitiba = math.trunc(len(df_boa_vista[df_boa_vista['Município'] == 'CURITIBA']) / len(df_boa_vista) * 100)
print(proporcao_boa_vista_curitiba,'% de atendimentos na UPA BOA VISTA são de atendimentos referentes a cidadãos curitibanos')

Quais os diagnósticos (abaixo há uma introdução sob CID's) dos pacientes de Colombo que são atendidos nesta UPA? 

In [ ]:
# 20 principais diagnósticos
df[ (df['Município'] == 'COLOMBO') & (df['Descrição da Unidade'] == 'UPA BOA VISTA') ]['Descrição do CID'].value_counts().reset_index(name='Atendimentos').head(20)

### Proporção de atendimentos por sexo

In [ ]:
f,ax=plt.subplots(1,2,figsize=(14,6))

# Todos
ax[0].set_title('Todos')
df['Sexo'].value_counts().plot(kind="pie", ax=ax[0])

# Internados
ax[1].set_title('Internados')
df_internacao['Sexo'].value_counts().plot(kind="pie", ax=ax[1])

** Proporcionalmente, homens geram mais internação. ** 

Há alguns anos atrás estava analisando dados do IBGE quando percebi que 85% de pessoas 'viúvas' eram mulheres. Uma das principais causas era justamente a falta de cuidado que homens tem em relação a saúde quando comparado com as mulheres.

### Distribuição por idade

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Todos')
sns.distplot(df['Idade'], bins=20, ax=ax[0]);

# Internados
ax[1].set_title('Internados')
sns.distplot(df_internacao['Idade'], bins=20, ax=ax[1]);

In [ ]:
f,ax=plt.subplots(1,1,figsize=(14,6))
sns.boxplot(x='Tipo de Unidade', y='Idade', hue='Desencadeou Internamento', data=df);

### Atendimentos por Faixa Etária

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Todos')
sns.countplot(data=df, x = 'Classificação Etária', order=['Criança','Adolescente','Adulto','Idoso'], ax=ax[0])

ax[1].set_title('Internados')
sns.countplot(data=df_internacao, x = 'Classificação Etária', order=['Criança','Adolescente','Adulto','Idoso'], ax=ax[1])

** Proporcionalmente, crianças geram menos internações e idosos geram mais internações. **

### Atendimentos por Turno de Atendimento

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Todos')
sns.countplot(data=df, x = 'Turno do Atendimento', order=['Dia','Noite','Madrugada'], ax=ax[0])

ax[1].set_title('Internados')
sns.countplot(data=df_internacao, x = 'Turno do Atendimento', order=['Dia','Noite','Madrugada'], ax=ax[1])

** Proporcionalmente, noite e madrugada geram mais internações, o que é esperado por se tratar de atendimentos de urgência. **

### Atendimentos por Unidade de Atendimento

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Todos')
sns.countplot(data=df, x = 'Tipo de Unidade', ax=ax[0])

ax[1].set_title('Internados')
sns.countplot(data=df_internacao, x = 'Tipo de Unidade', ax=ax[1])

** UPAs geram mais internações, o que é esperado por se tratar de Unidades de Pronto Atendimento **

### CID

CID significa 'Código Internacional de Doença' e mais informações podem ser obtidas em https://pt.wikipedia.org/wiki/Lista_de_c%C3%B3digos_da_CID-10

Vamos extrair a tabela com os códigos de doença do Wikipedia.

In [ ]:
df_tabelas_site = pd.read_html('https://pt.wikipedia.org/wiki/Classifica%C3%A7%C3%A3o_Estat%C3%ADstica_Internacional_de_Doen%C3%A7as_e_Problemas_Relacionados_com_a_Sa%C3%BAde#Codifica%C3%A7%C3%A3o', header=0)
df_tabelas_site[0]

Observando a tabela acima, podemos agrupar doenças de acordo com suas respectivas classes, utilizando a letra do CID, por exemplo, CIDs que começam pela letra 'L' são 'Doenças de pele e do tecido subcutâneo'

### Quais CID's geram mais atendimentos?

In [ ]:
df['Descrição do CID'].value_counts().reset_index(name='Atendimentos').head(10)

### Quais CID's geram mais internações?

In [ ]:
df_internacao['Descrição do CID'].value_counts().reset_index(name='Internações').head(10)

### Quais 'Categorias' de CID mais geram internações?

In [ ]:
df_internacao['Código do CID'].str[0].value_counts().head(10).plot(kind='bar')

In [ ]:
s_cid_todos = df['Código do CID'].str[0].value_counts() / len(df) * 100
s_cid_internados = df_internacao['Código do CID'].str[0].value_counts() / len(df_internacao) * 100

df_cid = pd.concat([s_cid_todos, s_cid_internados], axis=1)
df_cid.columns = ['Todos','Internados']

df_cid['Categoria'] = df_cid.index
df_cid.fillna(0, inplace=True)
df_cid

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,5))

# Todos
ax[0].set_title('Todos')
sns.barplot(data=df_cid, x='Categoria', y='Todos', ax=ax[0])

#Internados
ax[1].set_title('Internados')
sns.barplot(data=df_cid, x='Categoria', y='Internados', ax=ax[1])

** Proporcionalmente, CIDs da categoria J geram bastante internações, enquanto CIDs da categoria Z não geram internações. ** 

Categoria 'J': 'Doenças do aparelho respiratório'.
Categoria 'Z': 'Fatores que influenciam o estado de saúde e o contato com os serviços de saúde'.

A categoria 'R' se destaca em ambos os gráficos mas é inespecífica, geralmente por sintomas, sinais e achados anormais em exames laboratoriais que requerem investigação.


### Atendimentos por Solicitação de Exames e Encaminhamento ao Especialista

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,16))

# Solicitação de Exames - Todos
ax[0,0].set_title('Todos')
df['Solicitação de Exames'].value_counts().plot(kind="pie", ax=ax[0,0])

# Solicitação de Exames - Internados
ax[0,1].set_title('Internados')
df_internacao['Solicitação de Exames'].value_counts().plot(kind="pie", ax=ax[0,1])

# Encaminhamento para Especialista - Todos
ax[1,0].set_title('Todos')
df['Encaminhamento para Atendimento Especialista'].value_counts().plot(kind="pie", ax=ax[1,0])

# Encaminhamento para Especialista - Internados
ax[1,1].set_title('Internados')
df_internacao['Encaminhamento para Atendimento Especialista'].value_counts().plot(kind="pie", ax=ax[1,1])

** É raro ter 'Encaminhamento para Especialista' se a decisão for para internação. **

In [ ]:
dias_semana = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

f,ax=plt.subplots(2,2,figsize=(18,12))

# Todos
ax[0,0].set_title('Todos')
sns.countplot(data=df, x = 'Dia da Semana Atendimento Descricao', order=dias_semana, ax=ax[0,0])

ax[0,1].set_title('Internados')
sns.countplot(data=df_internacao, x = 'Dia da Semana Atendimento Descricao', order=dias_semana, ax=ax[0,1])

ax[1,0].set_title('Todos')
df['Fim de Semana'].value_counts().plot(kind="pie", ax=ax[1,0])

# Encaminhamento para Especialista - Internados
ax[1,1].set_title('Internados')
df_internacao['Fim de Semana'].value_counts().plot(kind="pie", ax=ax[1,1])

** Como esperado, o volume de internações aumenta durante os fins de semana **

### Quais bairros de Curitiba geram mais atendimentos?

In [ ]:
df_curitiba = df[df['Município'] == 'CURITIBA']

In [ ]:
# 20 principais bairros
df_curitiba['Bairro'].value_counts().head(20)

'Cidade Industrial' tem um maior número de atendimentos. Uma possibilidade futura é cruzar estas informações com o número de habitantes do bairro assim como outros indicadores sociais.